### In this notebook, we run simulations for producing output files for Gephi to do graph analysis without using SVD.

In [1]:
import pandas as pd
import numpy as np
import random
from random import shuffle, randrange
from collections import defaultdict
import utilities

import matplotlib.pyplot as plt
import seaborn as sns

import json

import AgentClass
import const
import networkx as nx
from config import num_agents,number_of_bits
from scipy import stats
from collections import defaultdict
import copy

In [2]:
%matplotlib inline

In [3]:
random.seed(1)
np.random.seed(1)

In [4]:
def get_tau_distr():
        lower = 0
        upper = 1
        mu = 0.5
        sigma = 0.1
        N = 1000
        
        samples = stats.truncnorm.rvs(
          (lower-mu)/sigma,(upper-mu)/sigma,loc=mu,scale=sigma,size=N)
        
        return samples

In [5]:
import numpy as np
import colorsys

# https://stackoverflow.com/a/9701141/5916727
def get_colors(num_colors):
    colors=[]
    for i in np.arange(0., 360., 360. / num_colors):
        hue = i/360.
        lightness = (50 + np.random.rand() * 10)/100.
        saturation = (90 + np.random.rand() * 10)/100.
        colors.append(colorsys.hls_to_rgb(hue, lightness, saturation))
    return colors

In [6]:
c = get_colors(1024)
t = [tuple(int(k*255) for k in i) for i in c ]
color_codes = {ind:{'r':i[0], 'g':i[1], 'b':i[2], 'a': 0} for ind, i in enumerate(t)}

In [7]:
def create_environment():
    list_agents = []
    tau_distr = get_tau_distr()
    
    for i in range(num_agents):
        in_state = [first_bit.pop(), second_bit.pop(), third_bit.pop(), fourth_bit.pop(), fifth_bit.pop()
                   , sixth_bit.pop(), seventh_bit.pop(), eigth_bit.pop(), ninth_bit.pop(), tenth_bit.pop()]
        a = AgentClass.Agent(name='agent{}'.format(i), tau=random.choice(tau_distr), initial_state=in_state)
        list_agents.append(a)
        
    # create network
    G = nx.newman_watts_strogatz_graph(num_agents, 10, 0.5, seed= 0)
#    nx.draw(G, with_labels=True, font_weight='bold') # to draw agents
    df = nx.to_pandas_adjacency(G, dtype=int)
    
    tmp_edges = df.apply(lambda row: row.to_numpy().nonzero()).to_dict()
    edges = {k: v[0].tolist() for k, v in tmp_edges.items()}
    
    # make random connections with agents
    for k, v in edges.items():
        for ngh in v:
            list_agents[k].add_neighbors(list_agents[ngh])
            
    return list_agents

In [8]:
def get_network_df(list_agents):
    network_df = pd.DataFrame({'Agent Name':[], 'Neighbors':[]})
    for agt in list_agents:
        neighbors = agt.get_neighbors_name()
        network_df = network_df.append({'Agent Name':agt.name, 
                                        'Neighbors':neighbors}, ignore_index=True)
    return network_df

Half of the agent were assigned 1's and other half 0's for each k bit knowledge

In [9]:
first_bit = [0 for i in range(num_agents//2)] + [1 for i in range(num_agents//2)]
second_bit = [0 for i in range(num_agents//2)] + [1 for i in range(num_agents//2)]
third_bit = [0 for i in range(num_agents//2)] + [1 for i in range(num_agents//2)]
fourth_bit = [0 for i in range(num_agents//2)] + [1 for i in range(num_agents//2)]
fifth_bit = [0 for i in range(num_agents//2)] + [1 for i in range(num_agents//2)]
sixth_bit = [0 for i in range(num_agents//2)] + [1 for i in range(num_agents//2)]
seventh_bit = [0 for i in range(num_agents//2)] + [1 for i in range(num_agents//2)]
eigth_bit = [0 for i in range(num_agents//2)] + [1 for i in range(num_agents//2)]
ninth_bit = [0 for i in range(num_agents//2)] + [1 for i in range(num_agents//2)]
tenth_bit = [0 for i in range(num_agents//2)] + [1 for i in range(num_agents//2)]

shuffle(first_bit)
shuffle(second_bit)
shuffle(third_bit)
shuffle(fourth_bit)
shuffle(fifth_bit)
shuffle(sixth_bit)
shuffle(seventh_bit)
shuffle(eigth_bit)
shuffle(ninth_bit)
shuffle(tenth_bit)

In [10]:
def run_simulation(alpha, coh_matrix, list_agents, end_time):
    
    d = []
    selected_time_stamps = [0,24, 49, 74, 99]
    for t in range(end_time):    
        # compute next state for all agents
        for agt in list_agents:
            agt.update_knowledge(alpha, coh_matrix) 
         
        # keep record of current record and all other values
        for agt in list_agents:
            row = {'Agent_Name':agt.name,
                   'Agent_Dissonance':agt.dissonance_lst,
                   'Time':t,
                   'Current State':agt.knowledge_state,
                   'Next State':agt.next_state}
            
            d.append(row)
        
        agent_states = {agent.name: utilities.bool2int(agent.knowledge_state) for agent in agents_list}

        if t in selected_time_stamps:
            G=nx.Graph()

            for agent in list_agents:
                agent_num = int(''.join([ch for ch in agent.name if ch.isdigit()]))
                agent_neighbors = [(int(''.join([ch for ch in neighbor_name if ch.isdigit()])), agent_states[neighbor_name]) for neighbor_name in agent.neighbors]
                nghbrs = [int(''.join([ch for ch in neighbor_name if ch.isdigit()])) for neighbor_name in agent.neighbors]
                agent_state = utilities.bool2int(agent.knowledge_state)
                G.add_node(agent_num)
                G.node[agent_num]['viz'] = {'color': color_codes[agent_state]}
                G.node[agent_num]['state'] = agent_state

                for n_agent in nghbrs:
                    G.add_edge(agent_num, n_agent)

            nx.write_gexf(G, "poster_outputs/gephi/network_alpha_{}_t_{}.gexf".format(alpha,t))
            
        # now update all agents next state with computed next state
        for agt in list_agents:
            agt.knowledge_state = agt.next_state
            agt.next_state = None
            agt.dissonance_lst = None
            
    return pd.DataFrame(d)

In [11]:
kbits_dict = defaultdict(list)

for i in range(1024):
    kbits_dict['State'].append(i)
    kbits_dict['binary'].append('{0:010b}'.format(i))
    
    kbits_dict['binary_list'].append(list('{0:010b}'.format(i)))

kbits_mapper_df = pd.DataFrame(kbits_dict)
kbits_mapper_df.set_index('State', inplace=True)
kbits_mapper_df.drop('binary', inplace=True, axis=1)
kbits_mapper_df_dict = kbits_mapper_df.to_dict(orient='index')

In [12]:
random.seed(1)

end_simulation_time = 100
alphas = [0,.25,.5,.75,1.0]
    
# first create environment
agents_list = create_environment()
    
# get network of the agents
agent_network_df = get_network_df(agents_list) 
    
results = {}
    
# for saving
# agent_network_df.to_json('simulations_new/test_network.json',orient='records', lines=True)
        
results['seed'] = 1
        
constants = const.Constants()
coherence_matrix = constants.get_coh_matrix().tolist()
results['coherence_matrix'] = coherence_matrix
        
    
results['alphas'] = defaultdict(list)
all_df = pd.DataFrame()
# run simulation
for alpha in alphas:
    record_df = None
    record_df = run_simulation(alpha, coherence_matrix, copy.deepcopy(agents_list), end_simulation_time)
    record_df['Current_Knowledge_State'] =  record_df['Current State']
    record_df['Current State'] = record_df['Current State'].apply(lambda row: utilities.bool2int(row))
    record_df['Next State'] = record_df['Next State'].apply(lambda row: utilities.bool2int(row))
    record_df['alpha'] = str(alpha)
    all_df = all_df.append(record_df)
    df = record_df.groupby(['Current State', 'Next State']).size().to_frame('Count').reset_index()

In [13]:
t_df= all_df[all_df['alpha']=='0']
cross_df = pd.crosstab(t_df['Current State'], t_df['Next State'])

This is for mapping knowledge state transitions graph with weight and color

In [27]:
for a in alphas:
    G = nx.DiGraph()
    x = all_df[all_df['alpha']==str(a)].groupby(['Current State', 'Next State']).size().to_frame('Count')
    x.reset_index(inplace=True)
    for _, row in x.iterrows():
        curr_state = row['Current State']

        G.add_node(curr_state)
        G.node[curr_state]['viz'] = {'color': color_codes[curr_state]}
        G.add_edge(curr_state, row['Next State'], weight=row['Count'])

    nx.write_gexf(G, "poster_outputs/gephi/knowledge_{}.gexf".format(a))